In [203]:
from bs4 import BeautifulSoup
import requests
import csv
product = input("what do you want to find? ")
product = product.replace(" ", "+")
listing = []
custom_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36', 
    'Accept-language': 'en-US,en;q=0.7'
}

In [199]:
#opening amazon and grabbing all the items
custom_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36', 
    'Accept-language': 'en-US,en;q=0.7'
}
amz = 'https://www.amazon.ca/s?k=' + product
res = requests.get(amz, headers= custom_header)
soup = BeautifulSoup(res.content, "html.parser")
print(amz)

https://www.amazon.ca/s?k=iem


In [204]:
#Grabs info straight from search for amazon
items = soup.find_all("div", attrs={"data-component-type": "s-search-result"})
for prod in items:
    link = "https://www.amazon.ca" + prod.find("a", attrs={"class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"}).get("href")
    name = prod.find("span", attrs={"class": "a-size-base-plus a-color-base a-text-normal"}).text.strip()
    if prod.find("span", attrs={"class": "a-icon-alt"}) is None:
        rating = "No reviews"
    else:
        rating = prod.find("span", attrs={"class": "a-icon-alt"}).text.strip()
        reviews = prod.find("span", attrs={"class": "a-size-base s-underline-text"}).text.strip()

    if prod.find("span", attrs={"class": "a-offscreen"}) is None:
        price = "Not in stock"
    else:
        price = prod.find("span", attrs={"class": "a-offscreen"}).text.strip()[1:]
    listing.append({"price": float(price), "rating": rating, "reviews": reviews,"name": name, "link": link})
listing

[{'price': '65.99',
  'rating': '4.3 out of 5 stars',
  'reviews': '14,591',
  'name': 'Linsoul KZ ZS10 Pro, 4BA+1DD 5 Driver in Ear Monitor, HiFi Wired Earbuds, Gaming Earbuds, Hybrid IEM Earphones with Stainless Steel Faceplate, Recessed 2 Pin Detachable Cable (Without Mic, Black)',
  'link': 'https://www.amazon.ca/sspa/click?ie=UTF8&spc=MTozMjYwNzE1Mjc4MjQwOTk0OjE3MjcyODQ1MTY6c3BfYXRmOjMwMDE2NzYxMzA3OTcwMjo6MDo6&url=%2FKZ-ZS10-Linsoul-Earphones-Detachable%2Fdp%2FB07QKYTGH9%2Fref%3Dsr_1_1_sspa%3Fdib%3DeyJ2IjoiMSJ9.rlYvWYPzLVoebPxABPo0JV4jNVqFVVB-8aC99JBflqWWBAMqZyzzOc6qxzWuobxeabCk6dnp7_U8NKyEbfz1b6rV3WuFIQNV904RY0w3J5IB10Pyt5G9q2kRIMpUvmD4x9wqNdi8l6L8QVI9RD6WTkViGRxmS6xTHB42317WbAgl80j6ISw7Qww_5BSZrGvLEtGqICxF7t06wPZirdS1CzYhKguDcLXGAWTTZ-y8LizFLaot3ZPqs8eXwtXgN2oO1_GligNsOfbJxt2IFY07lxPfhj0f3MnjQgnp-mdkjjo.jQBvcV4OJJqN4tLgdb0a_xqOaZLtQJvgIzkr_8_otP4%26dib_tag%3Dse%26keywords%3Diem%26qid%3D1727284516%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1'},
 {'price': '179.99

In [201]:
#Put all data into a csv, consider filtering out the ones that have no price
with open("data.csv", 'w', newline='') as csvf:
    fields = ["price", "rating", "reviews", "name", "link"]
    writer = csv.DictWriter(csvf, fieldnames= fields)
    writer.writeheader()
    writer.writerows(listing)